In [1]:
import pandas as pd
import glob
import numpy as np
from scipy import stats
from sklearn import cluster
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

In [46]:
df_cnv = pd.read_csv('./CNV_Marcotte.csv')

In [47]:
df_cnv

,Unnamed: 0,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2MP1,A3GALT2,...,ZNF177,ZNF223,ZNF286A,ZNF33B,ZNF668,ZNF709,ZNF747,ZNF763,ZNF788,ZNF8
0,184A1,0.0178,0.0178,0.0328,0.0188,0.0188,0.0188,0.018800,0.0188,0.0211,...,0.0091,0.0157,0.0326,0.0218,0.0131,0.009100,0.0131,0.0091,0.0091,0.0178
1,184B5,-0.0197,-0.0197,-0.0027,-0.0236,-0.0236,-0.0236,-0.023600,-0.0236,-0.0302,...,-0.0286,-0.0243,0.3041,-0.0203,-0.0199,-0.028600,-0.0199,-0.0286,-0.0286,-0.0197
2,MPE600,0.0269,0.0269,0.0359,0.0375,0.0375,0.0375,0.037500,0.0375,0.0222,...,0.0205,0.0269,0.0155,0.0390,0.0175,0.020500,0.0175,0.0205,0.0205,0.0269
3,AU565,-0.0293,-0.0293,-0.1147,0.0547,0.0547,0.0547,0.054700,0.0547,-0.0876,...,-0.3056,-0.0293,-0.1385,-0.0737,0.0459,-0.305600,0.0459,-0.3056,-0.3056,-0.0293
4,BT20,0.0985,0.0985,-0.1870,0.1015,0.1015,0.1015,0.101500,0.1015,-0.0962,...,-0.1012,0.0827,0.0953,0.0582,-0.1281,-0.101200,-0.1281,-0.1012,-0.1012,0.0985
5,BT474,-0.0189,-0.0189,0.0909,0.0641,0.0641,0.0641,0.064100,0.0641,-0.1540,...,-0.0645,-0.0378,-0.1227,0.0169,-0.0521,-0.064500,-0.0521,-0.0645,-0.0645,-0.0189
6,BT483,-0.0046,-0.0046,0.0053,0.0806,0.0806,0.0806,0.080600,0.0806,-0.3011,...,-0.3774,-0.0955,-0.3362,-0.0049,-0.0239,-0.377400,-0.0239,-0.3774,-0.3774,-0.0046
7,BT549,-0.0238,-0.0238,0.1735,-0.1491,-0.1491,-0.1491,-0.149100,-0.1491,0.0303,...,-0.0482,-0.0454,0.0632,0.0944,-0.0317,-0.048200,-0.0317,-0.0482,-0.0482,-0.0238
8,CAL120,0.2140,0.2140,0.3035,0.2560,0.2560,0.2560,0.256000,0.2560,0.0882,...,0.0581,0.0634,-0.0185,0.0992,0.0624,0.058100,0.3274,0.0581,0.0581,0.2140
9,CAL148,0.0230,0.0230,0.0145,0.0136,0.0136,0.0136,0.013600,0.0136,0.0259,...,0.0079,0.0230,0.0549,0.0276,0.1841,0.007900,0.1841,0.0079,0.0079,0.0230


In [9]:
df_rna = pd.read_csv('./RNAseq_Marcotte.csv')

In [10]:
df_rna

,Unnamed: 0,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2MP1,A3GALT2,...,ZNF223,ZNF286A,ZNF33B,ZNF668,ZNF709,ZNF747,ZNF763,ZNF788,ZNF8,Cellline
0,184A1,-3.113,-3.322,-3.322,-2.914,-1.751,5.767,-3.322,-2.457,-3.322,...,0.553,3.622,2.744,1.727,-0.929,1.649,-2.487,-3.322,0.376,184A1
1,184B5,2.298,2.425,-3.322,-3.322,-3.033,4.692,-3.322,-2.050,-3.322,...,0.172,4.215,3.622,-0.347,-2.281,0.554,-3.322,-3.322,2.173,184B5
2,MPE600,-0.131,0.975,-3.322,5.612,-2.174,-2.902,-3.322,-2.126,-3.322,...,1.001,2.590,2.448,-0.505,-0.645,1.322,0.684,0.538,2.380,MPE600
3,AU565,0.423,1.340,-3.322,-0.488,-2.355,3.008,-3.322,-1.053,-3.322,...,0.095,3.073,2.245,2.341,-3.322,-0.101,-3.322,-3.322,0.728,AU565
4,BT20,0.904,0.723,-3.322,-3.322,-1.706,-0.715,-3.322,-1.258,-3.322,...,1.496,3.496,2.199,0.332,-0.733,2.143,-0.296,-3.322,1.167,BT20
5,BT474,-3.317,-1.027,-3.322,-3.322,-1.701,0.087,-3.322,0.254,-3.322,...,-3.322,3.207,2.007,0.955,0.531,1.626,2.004,-0.178,2.161,BT474
6,BT483,1.509,1.613,-3.322,-2.670,-1.908,-1.166,-3.322,-1.633,-3.322,...,1.817,1.590,1.953,0.810,-0.758,2.197,-0.042,-0.885,1.524,BT483
7,BT549,2.746,2.592,-3.322,-3.322,-3.322,-3.322,-3.322,-0.244,-3.322,...,0.501,5.046,1.622,1.768,0.245,1.385,0.058,1.991,1.762,BT549
8,CAL120,2.519,3.501,-3.322,-0.350,-1.350,-3.322,-3.322,-2.281,-3.322,...,2.310,3.649,2.500,0.582,0.327,2.865,1.199,1.821,2.956,CAL120
9,CAL148,-0.776,1.784,-3.322,-3.322,-0.957,2.295,-3.322,-0.830,-3.322,...,-3.322,4.219,2.244,2.117,0.492,2.772,1.226,1.609,2.047,CAL148


In [44]:
df_prot = pd.read_csv('./Proteomics_log2FC_usefuldata_genemat.csv')

In [45]:
df_prot

,Cellline,NUDT4B,CYP2D7,CYP2D6,SMIM26,TRAV16,GATD3B,GATD3A,MMP24OS,RBM47,...,FAP,MAN2A2,SESN1,SLC2A3,SLC2A14,LTO1,R3HCC1L,SLC37A2,BTBD10,MAPK8IP3
0,AU565,2.280948,0.534789,0.534789,0.666483,1.217341,1.269833,1.269833,0.985800,7.289513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BT20,-0.336647,0.214997,0.214997,-1.476875,NaN,0.410916,0.410916,NaN,6.718455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BT474,2.289154,0.147820,0.147820,-0.998707,0.729033,1.330236,1.330236,1.936845,6.772977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BT483,-1.238482,-0.376452,-0.376452,NaN,0.828461,0.180556,0.180556,-1.032499,4.743943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BT549,-0.288732,0.093583,0.093583,NaN,2.267953,-0.481540,-0.481540,-0.065824,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CAL120,-0.047594,-0.305440,-0.305440,0.291513,-1.539768,-0.158307,-0.158307,2.351704,-0.926077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CAL148,1.797410,-0.413290,-0.413290,0.256808,2.071185,1.175748,1.175748,-2.778116,6.640482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CAL51,2.540174,0.064228,0.064228,-0.133667,-0.526644,-1.576618,-1.576618,-4.320141,5.080582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CAL851,0.668501,-0.581482,-0.581482,0.025001,-1.186618,-1.015613,-1.015613,-1.678742,5.629856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CAMA1,2.385310,0.217660,0.217660,-0.999951,2.886357,0.689916,0.689916,-3.826809,6.390914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
